In [1]:
# importing libraries
import pandas as pd
import numpy as np
from scipy import stats
from scipy.stats import norm, skew, boxcox_normmax
pd.set_option('display.max_columns', None)
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
#import geopandas as gpd
from datetime import datetime, timedelta
from sklearn import preprocessing 
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold, cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_squared_log_error
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

In [2]:
data = pd.read_csv('learningSet.csv')
data.shape

(95412, 481)

In [3]:
# checking targets
data.TARGET_B.value_counts()

0    90569
1     4843
Name: TARGET_B, dtype: int64

In [4]:
data.TARGET_D.value_counts()

0.00      90569
10.00       941
15.00       591
20.00       577
5.00        503
          ...  
101.00        1
4.50          1
55.00         1
10.70         1
13.92         1
Name: TARGET_D, Length: 71, dtype: int64

In [5]:
null_percent_df = pd.DataFrame(data.isna().sum()/len(data)).reset_index()
null_percent_df.columns=['column_name', 'per_of_null_values']
null_percent_df

,column_name,per_of_null_values
0,ODATEDW,0.000000
1,OSOURCE,0.000000
2,TCODE,0.000000
3,STATE,0.000000
4,ZIP,0.000000
...,...,...
476,MDMAUD_R,0.000000
477,MDMAUD_F,0.000000
478,MDMAUD_A,0.000000
479,CLUSTER2,0.001383


In [6]:
columns_above_threshold = null_percent_df[null_percent_df['per_of_null_values']>0.25]
columns_above_threshold

,column_name,per_of_null_values
23,NUMCHLD,0.870184
26,WEALTH1,0.468830
28,MBCRAFT,0.553955
29,MBGARDEN,0.553955
30,MBBOOKS,0.553955
...,...,...
451,RAMNT_20,0.917327
452,RAMNT_21,0.900296
453,RAMNT_22,0.781233
454,RAMNT_23,0.917631


In [7]:
to_drop_list = list(columns_above_threshold['column_name'])

In [8]:
print(to_drop_list)

['NUMCHLD', 'WEALTH1', 'MBCRAFT', 'MBGARDEN', 'MBBOOKS', 'MBCOLECT', 'MAGFAML', 'MAGFEM', 'MAGMALE', 'PUBGARDN', 'PUBCULIN', 'PUBHLTH', 'PUBDOITY', 'PUBNEWFN', 'PUBPHOTO', 'PUBOPP', 'WEALTH2', 'ADATE_5', 'ADATE_10', 'ADATE_13', 'ADATE_15', 'ADATE_17', 'ADATE_19', 'ADATE_20', 'ADATE_21', 'ADATE_22', 'ADATE_23', 'ADATE_24', 'RDATE_3', 'RDATE_4', 'RDATE_5', 'RDATE_6', 'RDATE_7', 'RDATE_8', 'RDATE_9', 'RDATE_10', 'RDATE_11', 'RDATE_12', 'RDATE_13', 'RDATE_14', 'RDATE_15', 'RDATE_16', 'RDATE_17', 'RDATE_18', 'RDATE_19', 'RDATE_20', 'RDATE_21', 'RDATE_22', 'RDATE_23', 'RDATE_24', 'RAMNT_3', 'RAMNT_4', 'RAMNT_5', 'RAMNT_6', 'RAMNT_7', 'RAMNT_8', 'RAMNT_9', 'RAMNT_10', 'RAMNT_11', 'RAMNT_12', 'RAMNT_13', 'RAMNT_14', 'RAMNT_15', 'RAMNT_16', 'RAMNT_17', 'RAMNT_18', 'RAMNT_19', 'RAMNT_20', 'RAMNT_21', 'RAMNT_22', 'RAMNT_23', 'RAMNT_24']


In [9]:
data['RDATE_3'].isna().sum()

95170

In [10]:
data['RAMNT_3'].isna().sum()

95170

In [11]:
l=['RDATE_3','RAMNT_3','WEALTH1','WEALTH2']
for col in l:
    to_drop_list.remove(col)
to_drop_list

['NUMCHLD',
 'MBCRAFT',
 'MBGARDEN',
 'MBBOOKS',
 'MBCOLECT',
 'MAGFAML',
 'MAGFEM',
 'MAGMALE',
 'PUBGARDN',
 'PUBCULIN',
 'PUBHLTH',
 'PUBDOITY',
 'PUBNEWFN',
 'PUBPHOTO',
 'PUBOPP',
 'ADATE_5',
 'ADATE_10',
 'ADATE_13',
 'ADATE_15',
 'ADATE_17',
 'ADATE_19',
 'ADATE_20',
 'ADATE_21',
 'ADATE_22',
 'ADATE_23',
 'ADATE_24',
 'RDATE_4',
 'RDATE_5',
 'RDATE_6',
 'RDATE_7',
 'RDATE_8',
 'RDATE_9',
 'RDATE_10',
 'RDATE_11',
 'RDATE_12',
 'RDATE_13',
 'RDATE_14',
 'RDATE_15',
 'RDATE_16',
 'RDATE_17',
 'RDATE_18',
 'RDATE_19',
 'RDATE_20',
 'RDATE_21',
 'RDATE_22',
 'RDATE_23',
 'RDATE_24',
 'RAMNT_4',
 'RAMNT_5',
 'RAMNT_6',
 'RAMNT_7',
 'RAMNT_8',
 'RAMNT_9',
 'RAMNT_10',
 'RAMNT_11',
 'RAMNT_12',
 'RAMNT_13',
 'RAMNT_14',
 'RAMNT_15',
 'RAMNT_16',
 'RAMNT_17',
 'RAMNT_18',
 'RAMNT_19',
 'RAMNT_20',
 'RAMNT_21',
 'RAMNT_22',
 'RAMNT_23',
 'RAMNT_24']

In [12]:
data = data.drop(columns =to_drop_list)

In [13]:
data.shape

(95412, 413)

In [14]:
data['MAILCODE'].value_counts()

     94013
B     1399
Name: MAILCODE, dtype: int64

In [15]:
data['MAILCODE'] = data['MAILCODE'].apply(lambda x: x.replace(" ", "A"))
data['MAILCODE']

0        A
1        A
2        A
3        A
4        A
        ..
95407    A
95408    A
95409    A
95410    A
95411    A
Name: MAILCODE, Length: 95412, dtype: object

In [16]:
data['MAILCODE'].value_counts()

A    94013
B     1399
Name: MAILCODE, dtype: int64

In [17]:
data['RECINHSE'] = data['RECINHSE'].apply(lambda x: x.replace(" ", "A"))
data['RECINHSE'].value_counts()

A    88709
X     6703
Name: RECINHSE, dtype: int64

In [18]:
data = data.apply(lambda x: x.replace(" ", np.NaN))

In [19]:
pd.set_option("display.max_columns", None)
data.head()

,ODATEDW,OSOURCE,TCODE,STATE,ZIP,MAILCODE,PVASTATE,DOB,NOEXCH,RECINHSE,RECP3,RECPGVG,RECSWEEP,MDMAUD,DOMAIN,CLUSTER,AGE,AGEFLAG,HOMEOWNR,CHILD03,CHILD07,CHILD12,CHILD18,INCOME,GENDER,WEALTH1,HIT,DATASRCE,MALEMILI,MALEVET,VIETVETS,WWIIVETS,LOCALGOV,STATEGOV,FEDGOV,SOLP3,SOLIH,MAJOR,WEALTH2,GEOCODE,COLLECT1,VETERANS,BIBLE,CATLG,HOMEE,PETS,CDPLAY,STEREO,PCOWNERS,PHOTO,CRAFTS,FISHER,GARDENIN,BOATS,WALKER,KIDSTUFF,CARDS,PLATES,LIFESRC,PEPSTRFL,POP901,POP902,POP903,POP90C1,POP90C2,POP90C3,POP90C4,POP90C5,ETH1,ETH2,ETH3,ETH4,ETH5,ETH6,ETH7,ETH8,ETH9,ETH10,ETH11,ETH12,ETH13,ETH14,ETH15,ETH16,AGE901,AGE902,AGE903,AGE904,AGE905,AGE906,AGE907,CHIL1,CHIL2,CHIL3,AGEC1,AGEC2,AGEC3,AGEC4,AGEC5,AGEC6,AGEC7,CHILC1,CHILC2,CHILC3,CHILC4,CHILC5,HHAGE1,HHAGE2,HHAGE3,HHN1,HHN2,HHN3,HHN4,HHN5,HHN6,MARR1,MARR2,MARR3,MARR4,HHP1,HHP2,DW1,DW2,DW3,DW4,DW5,DW6,DW7,DW8,DW9,HV1,HV2,HV3,HV4,HU1,HU2,HU3,HU4,HU5,HHD1,HHD2,HHD3,HHD4,HHD5,HHD6,HHD7,HHD8,HHD9,HHD10,HHD11,HHD12,ETHC1,ETHC2,ETHC3,ETHC4,ETHC5,ETHC6,HVP1,HVP2,HVP3,HVP4,HVP5,HVP6,HUR1,HUR2,RHP1,RHP2,RHP3,RHP4,HUPA1,HUPA2,HUPA3,HUPA4,HUPA5,HUPA6,HUPA7,RP1,RP2,RP3,RP4,MSA,ADI,DMA,IC1,IC2,IC3,IC4,IC5,IC6,IC7,IC8,IC9,IC10,IC11,IC12,IC13,IC14,IC15,IC16,IC17,IC18,IC19,IC20,IC21,IC22,IC23,HHAS1,HHAS2,HHAS3,HHAS4,MC1,MC2,MC3,TPE1,TPE2,TPE3,TPE4,TPE5,TPE6,TPE7,TPE8,TPE9,PEC1,PEC2,TPE10,TPE11,TPE12,TPE13,LFC1,LFC2,LFC3,LFC4,LFC5,LFC6,LFC7,LFC8,LFC9,LFC10,OCC1,OCC2,OCC3,OCC4,OCC5,OCC6,OCC7,OCC8,OCC9,OCC10,OCC11,OCC12,OCC13,EIC1,EIC2,EIC3,EIC4,EIC5,EIC6,EIC7,EIC8,EIC9,EIC10,EIC11,EIC12,EIC13,EIC14,EIC15,EIC16,OEDC1,OEDC2,OEDC3,OEDC4,OEDC5,OEDC6,OEDC7,EC1,EC2,EC3,EC4,EC5,EC6,EC7,EC8,SEC1,SEC2,SEC3,SEC4,SEC5,AFC1,AFC2,AFC3,AFC4,AFC5,AFC6,VC1,VC2,VC3,VC4,ANC1,ANC2,ANC3,ANC4,ANC5,ANC6,ANC7,ANC8,ANC9,ANC10,ANC11,ANC12,ANC13,ANC14,ANC15,POBC1,POBC2,LSC1,LSC2,LSC3,LSC4,VOC1,VOC2,VOC3,HC1,HC2,HC3,HC4,HC5,HC6,HC7,HC8,HC9,HC10,HC11,HC12,HC13,HC14,HC15,HC16,HC17,HC18,HC19,HC20,HC21,MHUC1,MHUC2,AC1,AC2,ADATE_2,ADATE_3,ADATE_4,ADATE_6,ADATE_7,ADATE_8,ADATE_9,ADATE_11,ADATE_12,ADATE_14,ADATE_16,ADATE_18,RFA_2,RFA_3,RFA_4,RFA_5,RFA_6,RFA_7,RFA_8,RFA_9,RFA_10,RFA_11,RFA_12,RFA_13,RFA_14,RFA_15,RFA_16,RFA_17,RFA_18,RFA_19,RFA_20,RFA_21,RFA_22,RFA_23,RFA_24,CARDPROM,MAXADATE,NUMPROM,CARDPM12,NUMPRM12,RDATE_3,RAMNT_3,RAMNTALL,NGIFTALL,CARDGIFT,MINRAMNT,MINRDATE,MAXRAMNT,MAXRDATE,LASTGIFT,LASTDATE,FISTDATE,NEXTDATE,TIMELAG,AVGGIFT,CONTROLN,TARGET_B,TARGET_D,HPHONE_D,RFA_2R,RFA_2F,RFA_2A,MDMAUD_R,MDMAUD_F,MDMAUD_A,CLUSTER2,GEOCODE2
0,8901,GRI,0,IL,61081,A,NaN,3712,0,A,NaN,NaN,NaN,XXXX,T2,36,60.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,F,NaN,0,NaN,0,39,34,18,10,2,1,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,X,992,264,332,0,35,65,47,53,92,1,0,0,11,0,0,0,0,0,0,0,11,0,0,0,39,48,51,40,50,54,25,31,42,27,11,14,18,17,13,11,15,12,11,34,25,18,26,10,23,18,33,49,28,12,4,61,7,12,19,198,276,97,95,2,2,0,0,7,7,0,479,635,3,2,86,14,96,4,7,38,80,70,32,84,16,6,2,5,9,15,3,17,50,25,0,0,0,2,7,13,27,47,0,1,61,58,61,15,4,2,0,0,14,1,0,0,2,5,17,73,0.0,177.0,682.0,307,318,349,378,12883,13,23,23,23,15,1,0,0,1,4,25,24,26,17,2,0,0,2,28,4,51,1,46,54,3,88,8,0,0,0,0,0,0,4,1,13,14,16,2,45,56,64,50,64,44,62,53,99,0,0,9,3,8,13,9,0,3,9,3,15,19,5,4,3,0,3,41,1,0,7,13,6,5,0,4,9,4,1,3,10,2,1,7,78,2,0,120,16,10,39,21,8,4,3,5,20,3,19,4,0,0,0,18,39,0,34,23,18,16,1,4,0,23,0,0,5,1,0,0,0,0,0,2,0,3,74,88,8,0,4,96,77,19,13,31,5,14,14,31,54,46,0,0,90,0,10,0,0,0,33,65,40,99,99,6,2,10,7,9706,9606.0,9604.0,9603.0,9602.0,9601.0,9511.0,9510.0,9508.0,9506.0,9503.0,9501.0,L4E,S4E,S4E,S4E,S4E,S4E,S4E,S4E,S4E,S4E,S4E,S4E,S4E,S4E,S4E,S4E,S4E,S4E,S4E,S4E,S4E,S4E,S4E,27,9702,74,6,14,NaN,NaN,240.0,31,14,5.0,9208,12.0,9402,10.0,9512,8911,9003.0,4.0,7.741935,95515,0,0.0,0,L,4,E,X,X,X,39.0,C
1,9401,BOA,1,CA,91326,A,NaN,5202,0,A,NaN,NaN,NaN,XXXX,S1,14,46.0,E,H,NaN,NaN,NaN,M,6.0,M,9.0,16,3,0,15,55,11,6,2,1,NaN,NaN,NaN,9.0,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3611,940,998,99,0,0,50,50,67,0,0,31,6,4,2,6,4,14,0,0,2,0,1,

In [20]:
null_percent_df2 = pd.DataFrame(data.isna().sum()/len(data)).reset_index()
null_percent_df2.columns=['column_name', 'per_of_null_values']
null_percent_df2

,column_name,per_of_null_values
0,ODATEDW,0.000000
1,OSOURCE,0.009726
2,TCODE,0.000000
3,STATE,0.000000
4,ZIP,0.000000
...,...,...
408,MDMAUD_R,0.000000
409,MDMAUD_F,0.000000
410,MDMAUD_A,0.000000
411,CLUSTER2,0.001383


In [21]:
columns_above_threshold2 = null_percent_df2[null_percent_df2['per_of_null_values']>0.25]
columns_above_threshold2

,column_name,per_of_null_values
6,PVASTATE,0.984719
10,RECP3,0.978860
11,RECPGVG,0.998805
12,RECSWEEP,0.983052
17,AGEFLAG,0.309689
19,CHILD03,0.987989
20,CHILD07,0.983587
21,CHILD12,0.981019
22,CHILD18,0.970161
25,WEALTH1,0.468830


In [22]:
to_drop_list2 = list(columns_above_threshold2['column_name'])
to_drop_list2

['PVASTATE',
 'RECP3',
 'RECPGVG',
 'RECSWEEP',
 'AGEFLAG',
 'CHILD03',
 'CHILD07',
 'CHILD12',
 'CHILD18',
 'WEALTH1',
 'SOLP3',
 'SOLIH',
 'MAJOR',
 'WEALTH2',
 'GEOCODE',
 'COLLECT1',
 'VETERANS',
 'BIBLE',
 'CATLG',
 'HOMEE',
 'PETS',
 'CDPLAY',
 'STEREO',
 'PCOWNERS',
 'PHOTO',
 'CRAFTS',
 'FISHER',
 'GARDENIN',
 'BOATS',
 'WALKER',
 'KIDSTUFF',
 'CARDS',
 'PLATES',
 'LIFESRC',
 'PEPSTRFL',
 'RFA_5',
 'RFA_10',
 'RFA_13',
 'RFA_15',
 'RFA_17',
 'RFA_19',
 'RFA_20',
 'RFA_21',
 'RFA_22',
 'RFA_23',
 'RFA_24',
 'RDATE_3',
 'RAMNT_3']

In [23]:
l2=[ 'VETERANS', 'SOLIH','WEALTH1','WEALTH2']
for col in l2:
    to_drop_list2.remove(col)
to_drop_list2

['PVASTATE',
 'RECP3',
 'RECPGVG',
 'RECSWEEP',
 'AGEFLAG',
 'CHILD03',
 'CHILD07',
 'CHILD12',
 'CHILD18',
 'SOLP3',
 'MAJOR',
 'GEOCODE',
 'COLLECT1',
 'BIBLE',
 'CATLG',
 'HOMEE',
 'PETS',
 'CDPLAY',
 'STEREO',
 'PCOWNERS',
 'PHOTO',
 'CRAFTS',
 'FISHER',
 'GARDENIN',
 'BOATS',
 'WALKER',
 'KIDSTUFF',
 'CARDS',
 'PLATES',
 'LIFESRC',
 'PEPSTRFL',
 'RFA_5',
 'RFA_10',
 'RFA_13',
 'RFA_15',
 'RFA_17',
 'RFA_19',
 'RFA_20',
 'RFA_21',
 'RFA_22',
 'RFA_23',
 'RFA_24',
 'RDATE_3',
 'RAMNT_3']

In [24]:
data = data.drop(columns =to_drop_list2)
data.shape

(95412, 369)

In [25]:
Y = data[['TARGET_D', 'TARGET_B']]
Y.head()

,TARGET_D,TARGET_B
0,0.0,0
1,0.0,0
2,0.0,0
3,0.0,0
4,0.0,0


In [26]:
# categorical data
categorical = data.select_dtypes(np.object)
categorical.head(10)

,OSOURCE,STATE,ZIP,MAILCODE,NOEXCH,RECINHSE,MDMAUD,DOMAIN,CLUSTER,HOMEOWNR,GENDER,DATASRCE,SOLIH,VETERANS,RFA_2,RFA_3,RFA_4,RFA_6,RFA_7,RFA_8,RFA_9,RFA_11,RFA_12,RFA_14,RFA_16,RFA_18,RFA_2R,RFA_2A,MDMAUD_R,MDMAUD_F,MDMAUD_A,GEOCODE2
0,GRI,IL,61081,A,0,A,XXXX,T2,36,NaN,F,NaN,NaN,NaN,L4E,S4E,S4E,S4E,S4E,S4E,S4E,S4E,S4E,S4E,S4E,S4E,L,E,X,X,X,C
1,BOA,CA,91326,A,0,A,XXXX,S1,14,H,M,3,NaN,NaN,L2G,A2G,A2G,A2G,A1E,A1E,A1E,A1E,A1E,NaN,L1E,NaN,L,G,X,X,X,A
2,AMH,NC,27017,A,0,A,XXXX,R2,43,U,M,3,NaN,NaN,L4E,S4E,S4E,S4E,S4F,S4F,S4F,S4F,S4F,S4F,S4F,S4D,L,E,X,X,X,C
3,BRY,CA,95953,A,0,A,XXXX,R2,44,U,F,3,NaN,NaN,L4E,S4E,S4E,S4E,S4E,S4E,S4E,S4E,S4E,S4E,S4E,S2D,L,E,X,X,X,C
4,NaN,FL,33176,A,0,X,XXXX,S2,16,H,F,3,12,NaN,L2F,A2F,A2F,A1D,I2D,A1E,A1E,A1E,A1E,L3D,L3D,A2D,L,F,X,X,X,A
5,CWR,AL,35603,A,0,A,XXXX,T2,40,NaN,NaN,NaN,NaN,NaN,L1F,A1F,A1F,A1F,A2F,A2F,A2F,A2F,A2F,A2F,N3F,N2E,L,F,X,X,X,C
6,DRK,IN,46755,A,0,A,XXXX,T2,40,H,F,3,NaN,NaN,L1E,A1E,A1E,A1E,S2E,S2E,S3E,S3E,S3E,S2E,S3E,S2E,L,E,X,X,X,D
7,NWN,LA,70611,A,0,A,XXXX,T2,39,U,F,1,NaN,NaN,L3E,A3E,A3E,A3E,A3E,A3E,A3E,A2D,A2D,A1D,N2D,N2D,L,E,X,X,X,C
8,LIS,IA,51033,A,0,A,XXXX,R2,45,U,M,3,NaN,NaN,L1F,A1F,A1F,A1F,S2F,S2F,S2F,A1F,A1F,A1F,A1F,A1F,L,F,X,X,X,D
9,MSD,TN,37127-,A,0,A,XXXX,T1,35,NaN,M,NaN,NaN,NaN,L1F,A1F,A1F,A1F,A1F,A1F,A1F,A1F,A1F,NaN,A1E,A1E,L,F,X,X,X,B


In [27]:
# Check for null values in all the columns

categorical.isna().sum()

OSOURCE       928
STATE           0
ZIP             0
MAILCODE        0
NOEXCH          7
RECINHSE        0
MDMAUD          0
DOMAIN       2316
CLUSTER      2316
HOMEOWNR    22228
GENDER       2957
DATASRCE    21280
SOLIH       89212
VETERANS    84986
RFA_2           0
RFA_3        1950
RFA_4        2312
RFA_6        3557
RFA_7        8874
RFA_8        3511
RFA_9       11245
RFA_11      10422
RFA_12       8923
RFA_14      18867
RFA_16      20417
RFA_18      21263
RFA_2R          0
RFA_2A          0
MDMAUD_R        0
MDMAUD_F        0
MDMAUD_A        0
GEOCODE2      319
dtype: int64

In [28]:
# cleaning STATE column
data['STATE'].unique()

array(['IL', 'CA', 'NC', 'FL', 'AL', 'IN', 'LA', 'IA', 'TN', 'KS', 'MN',
       'UT', 'MI', 'MO', 'TX', 'OR', 'WA', 'WI', 'GA', 'OK', 'SC', 'KY',
       'MD', 'SD', 'NV', 'NE', 'AZ', 'VA', 'ND', 'AK', 'AR', 'NM', 'MT',
       'MS', 'AP', 'CO', 'AA', 'HI', 'ME', 'WY', 'ID', 'OH', 'NJ', 'MA',
       'VT', 'NY', 'PA', 'DC', 'AE', 'CT', 'DE', 'WV', 'RI', 'GU', 'AS',
       'NH', 'VI'], dtype=object)

In [29]:
df = pd.DataFrame(data['STATE'].value_counts().reset_index())
df.columns = ['state','count']

In [30]:
other_states = list(df[df['count'] < 2500]['state'])

In [31]:
print(other_states)

['TN', 'AZ', 'OR', 'MN', 'CO', 'SC', 'AL', 'KY', 'OK', 'LA', 'KS', 'IA', 'AR', 'MS', 'NV', 'NM', 'NE', 'UT', 'ID', 'MT', 'HI', 'SD', 'WY', 'AK', 'ND', 'AP', 'NY', 'VA', 'OH', 'PA', 'MD', 'NJ', 'MA', 'CT', 'AA', 'AE', 'ME', 'NH', 'VT', 'RI', 'VI', 'WV', 'GU', 'DE', 'DC', 'AS']


In [32]:
# function to assign states with count less than 2500 as other
def clean_state(x):
    if x in other_states:
        return 'other'
    else:
        return x

In [33]:
categorical ['STATE'] = list(map(clean_state, categorical ['STATE']))

In [34]:
categorical ['STATE']

0           IL
1           CA
2           NC
3           CA
4           FL
         ...  
95407    other
95408       TX
95409       MI
95410       CA
95411       NC
Name: STATE, Length: 95412, dtype: object

In [35]:
# working on column domain
categorical ['DOMAIN'].value_counts()

R2    13623
T2    12369
S1    11503
S2     8530
C2     8264
C1     6145
C3     5280
T1     4982
R3     4809
U1     4510
U2     3254
U3     2598
T3     2176
S3     1891
U4     1804
R1     1358
Name: DOMAIN, dtype: int64

In [36]:
categorical['DOMAIN'].isna().sum()

2316

In [37]:
# filling the null with mode
categorical['DOMAIN'] = categorical['DOMAIN'].fillna('R2')

In [38]:
categorical['DOMAIN'].isna().sum()

0

In [39]:
categorical['DOMAIN_A'] = list(map(lambda x: x[0], categorical['DOMAIN']))
categorical['DOMAIN_B'] = list(map(lambda x: x[1], categorical['DOMAIN']))

In [40]:
categorical = categorical.drop(columns =['DOMAIN'])

In [41]:
categorical['RFA_2'].value_counts()

L1F    30380
L1G    12384
L2F    10961
L3E     7767
L2E     4989
L4D     4914
L1E     4911
L2G     4595
L4E     3972
L3F     3523
L3D     2498
L4F     2100
L3G     1503
L4G      915
Name: RFA_2, dtype: int64

In [42]:
r_columns = []
for col_name in categorical.columns:
    if 'RFA' in col_name:
        r_columns.append(col_name)

In [43]:
r_columns

['RFA_2',
 'RFA_3',
 'RFA_4',
 'RFA_6',
 'RFA_7',
 'RFA_8',
 'RFA_9',
 'RFA_11',
 'RFA_12',
 'RFA_14',
 'RFA_16',
 'RFA_18',
 'RFA_2R',
 'RFA_2A']

In [44]:
y = categorical[r_columns]
y

,RFA_2,RFA_3,RFA_4,RFA_6,RFA_7,RFA_8,RFA_9,RFA_11,RFA_12,RFA_14,RFA_16,RFA_18,RFA_2R,RFA_2A
0,L4E,S4E,S4E,S4E,S4E,S4E,S4E,S4E,S4E,S4E,S4E,S4E,L,E
1,L2G,A2G,A2G,A2G,A1E,A1E,A1E,A1E,A1E,NaN,L1E,NaN,L,G
2,L4E,S4E,S4E,S4E,S4F,S4F,S4F,S4F,S4F,S4F,S4F,S4D,L,E
3,L4E,S4E,S4E,S4E,S4E,S4E,S4E,S4E,S4E,S4E,S4E,S2D,L,E
4,L2F,A2F,A2F,A1D,I2D,A1E,A1E,A1E,A1E,L3D,L3D,A2D,L,F
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95407,L1G,F1G,F1G,F1G,P1G,P1G,NaN,NaN,NaN,NaN,NaN,NaN,L,G
95408,L1F,NaN,NaN,NaN,P1F,P1F,NaN,NaN,NaN,NaN,NaN,NaN,L,F
95409,L3E,S4E,S4E,S3E,NaN,A2E,N3E,N3E,N3E,N3E,F1D,F1D,L,E
95410,L4F,S4F,A3F,S4F,S4F,S4F,S4F,S4F,S4F,S4F,S3F,S2F,L,F


In [45]:
y.isna().sum()

RFA_2         0
RFA_3      1950
RFA_4      2312
RFA_6      3557
RFA_7      8874
RFA_8      3511
RFA_9     11245
RFA_11    10422
RFA_12     8923
RFA_14    18867
RFA_16    20417
RFA_18    21263
RFA_2R        0
RFA_2A        0
dtype: int64

In [46]:
l3 = ['RFA_2','RFA_2A']
for item in l3:
        r_columns.remove(item)
r_columns


['RFA_3',
 'RFA_4',
 'RFA_6',
 'RFA_7',
 'RFA_8',
 'RFA_9',
 'RFA_11',
 'RFA_12',
 'RFA_14',
 'RFA_16',
 'RFA_18',
 'RFA_2R']

In [47]:
categorical = categorical.drop(columns= r_columns)
categorical.shape

(95412, 21)

In [50]:
categorical['CLUSTER'].isna().sum()

2316

In [48]:
categorical['CLUSTER'].value_counts()

40    3979
35    3624
36    3617
27    3553
24    3539
49    3315
12    3000
18    2897
13    2706
30    2607
39    2601
45    2515
43    2385
11    2378
51    2322
14    2246
41    2176
44    1953
16    1947
2     1942
21    1891
8     1828
10    1804
46    1800
28    1630
17    1621
20    1568
53    1441
3     1428
34    1358
42    1358
23    1324
31    1285
22    1282
25    1275
38    1200
15    1173
1     1140
7     1076
5     1014
37     972
26     947
47     868
29     859
32     795
48     787
50     776
9      770
6      603
33     593
4      561
19     497
52     270
Name: CLUSTER, dtype: int64

In [51]:
# replace null values inside the category 
categorical['CLUSTER'] = categorical['CLUSTER'].fillna('40')

In [52]:
categorical['HOMEOWNR'].isna().sum()

22228

In [53]:
categorical['HOMEOWNR'].value_counts()

H    52354
U    20830
Name: HOMEOWNR, dtype: int64

In [54]:
categorical['HOMEOWNR'] = categorical['HOMEOWNR'].fillna('H')

In [55]:
# Exclude the following variables by looking at the definitions. Create a new empty list called drop_list. We will append this list and then drop all the columns in this list later:
# OSOURCE - symbol definitions not provided, too many categories
#ZIP CODE - we are including state already
to_drop_list3 = []

In [56]:

null_percent_df3 = pd.DataFrame(categorical.isna().sum()/len(categorical)).reset_index()
null_percent_df3.columns=['column_name', 'per_of_null_values']
null_percent_df3

,column_name,per_of_null_values
0,OSOURCE,0.009726
1,STATE,0.000000
2,ZIP,0.000000
3,MAILCODE,0.000000
4,NOEXCH,0.000073
5,RECINHSE,0.000000
6,MDMAUD,0.000000
7,CLUSTER,0.000000
8,HOMEOWNR,0.000000
9,GENDER,0.030992


In [57]:
# Identify columns that over 85% missing values
columns_above_threshold3 = null_percent_df3[null_percent_df3['per_of_null_values']>0.85]
for col in columns_above_threshold3['column_name']:
    to_drop_list3.append(col)
to_drop_list3



['SOLIH', 'VETERANS']

In [58]:
#Remove those columns from the dataframe. in the activity we were asked not to remove these columns. so, I make a copy of the categorical and drop them from the copy
categorical_copy = categorical.copy()

In [60]:
categorical_copy = categorical_copy.drop(columns= to_drop_list3)

In [61]:
categorical_copy 

,OSOURCE,STATE,ZIP,MAILCODE,NOEXCH,RECINHSE,MDMAUD,CLUSTER,HOMEOWNR,GENDER,DATASRCE,RFA_2,RFA_2A,MDMAUD_R,MDMAUD_F,MDMAUD_A,GEOCODE2,DOMAIN_A,DOMAIN_B
0,GRI,IL,61081,A,0,A,XXXX,36,H,F,NaN,L4E,E,X,X,X,C,T,2
1,BOA,CA,91326,A,0,A,XXXX,14,H,M,3,L2G,G,X,X,X,A,S,1
2,AMH,NC,27017,A,0,A,XXXX,43,U,M,3,L4E,E,X,X,X,C,R,2
3,BRY,CA,95953,A,0,A,XXXX,44,U,F,3,L4E,E,X,X,X,C,R,2
4,NaN,FL,33176,A,0,X,XXXX,16,H,F,3,L2F,F,X,X,X,A,S,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95407,ASE,other,99504,A,0,A,XXXX,27,H,M,NaN,L1G,G,X,X,X,C,C,2
95408,DCD,TX,77379,A,0,A,XXXX,24,H,M,3,L1F,F,X,X,X,A,C,1
95409,MBC,MI,48910,A,0,A,XXXX,30,H,M,NaN,L3E,E,X,X,X,B,C,3
95410,PRV,CA,91320,A,0,X,XXXX,24,H,F,2,L4F,F,X,X,X,A,C,1


In [63]:
#Reduce the number of categories in the column GENDER. The column should only have either "M" for males, "F" for females, and "other" for all the rest
categorical_copy['GENDER'].value_counts()

F    51277
M    39094
U     1715
J      365
C        2
A        2
Name: GENDER, dtype: int64

In [65]:
other_gender = ['U', 'J', 'C', 'A']
def clean_gender(x):
    if x in other_gender:
        return 'other'
    else:
        return x

In [66]:
categorical_copy ['GENDER'] = list(map(clean_gender, categorical_copy ['GENDER']))
categorical ['GENDER'] = list(map(clean_gender, categorical ['GENDER']))

In [68]:
categorical_copy ['GENDER'].value_counts()

F        51277
M        39094
other     2084
Name: GENDER, dtype: int64

In [69]:
categorical_copy ['GENDER'].isna().sum()

2957

In [71]:
#Reduce the number of categories in the column GENDER. The column should only have either "M" for males, "F" for females, and "other" for all the rest

#Note that there are a few null values in the column. We will first replace those null values using the code below:
#print(categorical['GENDER'].value_counts())
#categorical['GENDER'] = categorical['GENDER'].fillna('F')

print(categorical_copy['GENDER'].value_counts())
categorical_copy['GENDER'] = categorical['GENDER'].fillna('F')

F        51277
M        39094
other     2084
Name: GENDER, dtype: int64


In [72]:
categorical_copy['GENDER'].isna().sum()

0